In [1]:
import sys, os, io
import json
import datetime
import torch
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.dag import lexicographical_topological_sort
import dagviz
from matplotlib import pyplot as plt
from PIL import Image
from copy import deepcopy
from tqdm import tqdm


from data.loader import LMDB_Folder_Dataset 
from data.data_utils import dict2tensor
from dp.graph_utils import compute_metadag_costs, metadag2vid, generate_metagraph, remove_nodes_from_graph
from dp.exact_dp import drop_dtw
from dp.dp_utils import compute_all_costs
from dp.visualization import (color_code, shape_code, color_code_hex,
                              color_values, plot_step_to_video_alignment, plot_alignment)

from eval.metrics import framewise_accuracy, IoU, recall_crosstask, IoU_class, Acc_class
# from data.ct_graph import get_orig_sample, get_graph_sample, get_inverse_sample, graph_annots, name_to_idx, ct_dataset
from data.ct_graph import init_dataset, init_graph, get_graph_sample, map_segmentation_to_graph_nodes

device = "cpu"

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
# setting up the data loader, graphs, and the text encoder

name_to_idx, ct_dataset = init_dataset()
graph_annots = init_graph(type='learned_parser')

removed edges


# Full evaluation

In [3]:
# evaluating step localization using graph grounding vs aligning with linear recipes

kp = 0.1
gamma = 10
use_negative = False

res = {'graph': [], 'lin': []}
for name in tqdm(list(name_to_idx.keys())):
    graph_sample = get_graph_sample(name, remove_absent_steps=False)
    
    # graph sample
    metadag = generate_metagraph(graph_sample['graph'])
    sorted_node_ids = list(lexicographical_topological_sort(metadag))
    idx2node = {idx: node_id for idx, node_id in enumerate(sorted_node_ids)}
    zx_costs, drop_costs = compute_metadag_costs(graph_sample, idx2node, gamma, keep_percentile=kp)
    _, labels = metadag2vid(zx_costs.numpy(), drop_costs.numpy(), metadag, idx2node, contiguous=True)
    
    mapped_labels = -np.ones_like(labels)
    for rec_id, cap_id in graph_sample['recipe_2_caption'].items():
        mapped_labels[labels == rec_id] = cap_id
        
    iou_cls = IoU_class(mapped_labels, graph_sample['gt_seg'])
    acc_cls = Acc_class(mapped_labels, graph_sample['gt_seg'], use_negative=use_negative)
    res['graph'].append((acc_cls, iou_cls))
    
    # lin sample
    zx_costs, drop_costs, _ = compute_all_costs(graph_sample, None, gamma, 'logit', kp, do_logsoftmax=True)
    labels = drop_dtw(zx_costs.numpy(), drop_costs.numpy(), contiguous=True, return_labels=True) - 1
    
    mapped_labels = -np.ones_like(labels)
    for rec_id, cap_id in graph_sample['recipe_2_caption'].items():
        mapped_labels[labels == rec_id] = cap_id
        
    iou_cls = IoU_class(mapped_labels, graph_sample['gt_seg'])
    acc_cls = Acc_class(mapped_labels, graph_sample['gt_seg'], use_negative=use_negative)
    res['lin'].append((acc_cls, iou_cls))
    
for name in res:
    acc = np.mean([r[0] for r in res[name]])
    iou = np.mean([r[1] for r in res[name]])
    print(f"{name} | Acc {acc}, IoU: {iou}")

  0%|                                                    | 0/339 [00:00<?, ?it/s]/user/n.dvornik/Projects/Release/Graph2Vid/data/ct_graph.py:57: FutureWarning: 'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  sample = ct_dataset[name_to_idx[name]]
100%|██████████████████████████████████████████| 339/339 [01:58<00:00,  2.87it/s]

graph | Acc 0.24872163906376085, IoU: 0.16618270980114894
lin | Acc 0.23615385526957994, IoU: 0.15923570899747638
